### TP3 - 2

2. O seguinte sistema dinâmico denota 4 inversores ($\,A, B, C, D\,$) que lêm um bit num canal input e escrevem num canal output uma transformação desse bit.


<img src="inversores.png" alt="autómato"/>


   1. Cada inversor tem um bit $s$ de estado, inicializado  com um valor aleatório.
   2. Cada inversor é regido pelas seguintes transformações 
  
     $$ \mathbf{invert}\mathtt(in,out) $$
     $$x \gets \mathsf{read}(\mathtt{in})$$
     $$s \gets \neg x\;\;\|\;\; s\gets s\oplus x$$             
     $$\mathsf{write}(\mathtt{out},s)$$
     
        iii. A escolha neste comando é sempre determinística; isto é, em cada inversor a escolha do comando a executar é sempre a mesma. Porém qual é essa escolha é determinada aleatoriamente na inicializarão do sistema.
        
        iii. O estado do sistema é um duplo definido pelos 4 bits $s$, e é inicializado com um vetor aleatório em $\{0,1\}^4\;$.
        
        iv. O sistema termina em ERRO quando o estado do sistema for $\,(0,0,0,0)\,$.


1. Construa um SFOTS que descreva este sistema e implemente este sistema, numa abordagem BMC (“bouded model checker”) num traço com $\,n\,$ estados.
    
2. Verifique se o sistema é seguro usando BMC, k-indução ou model checking com interpolantes.

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT
import itertools 
from random import random, choice

### Função genState bla bla bla

In [2]:
# vars = [s', 'x']
def genState(vars,s,i):
    state = {}
    for j in range(1, 5):
        for v in vars:
            if v not in state:
                state[v] = {}
            
            state[v][j] = Symbol(v+str(j)+'_'+s+str(i))
            
    #print(state)
    
            
    return state

In [3]:
'''
A = Symbol('a')
B = Symbol('b')
with Solver(name='z3')
    s.solve(Xor(A,B))
'''

"\nA = Symbol('a')\nB = Symbol('b')\nwith Solver(name='z3')\n    s.solve(Xor(A,B))\n"

#### Função init(state):

    state - Dicionário de variáveis de estado


   A função `init` tem como objetivo devolver um predicado do Solver que testa se é um possível estado inicial do programa, através do `state`, um dicionário de variáveis.

In [4]:
def init(state):
    #random_state = (Bool(choice([True, False])),Bool(choice([True, False])),Bool(choice([True, False])),Bool(choice([True, False]))) 
    
    random_state = (Bool(True), Bool(True), Bool(True), Bool(True))
    
    x_list = []
    print(state['x'])
    for v in state['x']:
        x_list.append( Iff( state['x'][v] , random_state[v-2]) )

    s_list = []
    for v in state['s']:
        s_list.append( Iff(state['s'][v], random_state[v-1]) )
    
    
    return And(
        And(x_list),
        And(s_list),
    )

#### Função trans(curr, prox):

    curr - Estado das variáveis no momento atual
    prox - Estado das variáveis no momento da próxima iteração
    
   A função `trans` tem como objetivo devolver um predicado do Solver, através dos três estados disponíveis, que teste se é possível transitar entre os estados possíveis.

In [5]:
def error(state):
    return Or( 
        And( [Iff( state['s'][i], Bool(False)) for i in range(1,5)] )
    )


def trans(curr, prox):
    zero_state = And( [Iff( curr['s'][i], Bool(False)) for i in range(1,5)] )
    
    # prox_x = prox_s
    # prox_s = Not(curr[x])  V Xor(curr[s], curr[x])
    magia = And(
        Not(zero_state),
        And([ Iff(prox['x'][i], prox['s'][i]) for i in range(1,5)]),
        And([ Iff(prox['s'][i], Not(And( curr['x'][i] , Not(Xor(curr['x'][i], curr['s'][i])))  )) for i in range(1,5)])
    )
    
    error = And(
        zero_state,
        And( [Iff(prox['x'][i], curr['x'][i]) for i in range(1,5)]),
        And( [Iff(prox['s'][i], curr['s'][i]) for i in range(1,5)]),
    )
    
    return Or(magia, error)
    

#### Função gera_traco(declare, init, trans, k, n, a, b)

    declare - Variáveis de estado
    init - Condições para o estado inicial
    trans - Função transição
    k - Valor do traço


A função `gera_traco` tem como objetivo imprimir o valor das variáveis à medida que vão percorrendo os estados, através das variáveis do estado e de um predicado que testa se um estado é inicial.

In [6]:
def gera_traco(vars, init, trans, error, k):
    with Solver(name="z3") as s:
    
        X = [genState(vars,'X',i) for i in range(k+1)]
        #print(X)
        I = init(X[0])
        print(I)
        Tks = [ trans(X[i],X[i+1]) for i in range(k) ]
        #print(Tks)
    
        if s.solve([I,And(Tks)]):
            for i in range(k):
                print("Estado:",i)
                for v in X[i]:
                    for k in range(1,5):
                        print("          ",v,'=',s.get_value(X[i][v][k]))
                print("----------------")
        else:
            print("unsat")
        
        
gera_traco(['s', 'x'], init, trans, error, 20)

{1: x1_X0, 2: x2_X0, 3: x3_X0, 4: x4_X0}
(((x1_X0 <-> True) & (x2_X0 <-> True) & (x3_X0 <-> True) & (x4_X0 <-> True)) & ((s1_X0 <-> True) & (s2_X0 <-> True) & (s3_X0 <-> True) & (s4_X0 <-> True)))


KeyError: -1